In [2]:
%%capture
%pip install redis rich tqdm

In [3]:
import redis
from rich.pretty import pprint

In [4]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)

[redis command docs](https://redis.io/docs/latest/commands/)

In [5]:
# grab item from sample_bicycle 
r.json().get("sample_bicycle:1001")

{'model': 'Racer',
 'brand': 'Speedster',
 'price': 320,
 'type': 'Road',
 'specs': {'material': 'carbon fiber', 'weight': 8},
 'description': 'The Racer is built for speed and performance on the road!',
 'addons': ['water bottle holder', 'bike lock'],
 'helmet_included': True,
 'condition': 'new'}

In [6]:
# iterate over sample_bicycle keys 
r.scan_iter("sample_bicycle:*")

<generator object ClusterDataAccessCommands.scan_iter at 0xffff8bb08970>

In [11]:
# iterate oversample_bicycle keys fetch brands and types
for key in r.scan_iter("sample_bicycle:*"):
    bicycle = r.json().get(key)
    # use set add to store brands/types just once
    r.sadd("sample_bicycle_index:brands",bicycle["brand"])
    r.sadd("sample_bicycle_index:types",bicycle["type"])

In [12]:
# get unique brands
r.smembers("sample_bicycle_index:types")

{b'BMX',
 b'City',
 b'Electric',
 b'Fat',
 b'Gravel',
 b'Hybrid',
 b'Kids',
 b'Leisure',
 b'Mountain',
 b'Road',
 b'Touring'}

In [21]:
# get unique brands 
# brands can be quite a few, we use iterative set scan to avoid blocking the server
pprint(list(r.sscan_iter("sample_bicycle_index:brands"))[:20])

[
│   b'Cyclopedic',
│   b'Tiny Trekkers',
│   b'Speedy Sprint',
│   b'Family Treasures',
│   b'Adventure Kids',
│   b'Extreme Trails',
│   b'Urban Trek',
│   b'Peak Riders',
│   b'Urban Roam',
│   b'Hand-Me-Down Bikes',
│   b'SkyHigh',
│   b'EcoRide',
│   b'Nomad',
│   b'Extreme',
│   b'Pathfinder',
│   b'Speedster',
│   b'Terrain Tamer',
│   b'Summit Seekers',
│   b'City Commute',
│   b'Summit Crushers'
]

In [55]:
# reset counter 
r.delete("sample_bicycle_index:brand_count")
r.delete("sample_bicycle_index:type_count")

# set brands and types indexes
for key in r.scan_iter("sample_bicycle:*"):
    bicycle = r.json().get(key)
    # use set add to store brands/types just once
    r.sadd("sample_bicycle_index:brand:" + bicycle["brand"], key)
    r.sadd("sample_bicycle_index:type:" + bicycle["type"], key)
    
    # increment multicounter
    r.hincrby("sample_bicycle_index:brand_count", bicycle["brand"], 1)
    r.hincrby("sample_bicycle_index:type_count", bicycle["type"], 1)

In [63]:
# get Gravel bicycles
r.smembers("sample_bicycle_index:type:Gravel")

{b'sample_bicycle:1006', b'sample_bicycle:1016', b'sample_bicycle:1032'}

In [64]:
# bicycles type: Kids, brand: Tiny Trekkers?
# use intersection ons type:Kids and brand:Tiny Trekkers
try:
    pprint(r.sinter(["sample_bicycle_index:type:Kids","sample_bicycle_index:brand:Tiny Trekkers"]))
except Exception as e:
    print("error",e)

error SINTER - all keys must map to the same key slot
